In [149]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import os
import csv

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import warnings
warnings.filterwarnings('ignore')

In [150]:
# Determine important landmarks for plank
landmarks = [
    'NOSE',
    'LEFT_SHOULDER',
    'RIGHT_SHOULDER',
    'LEFT_ELBOW',
    'RIGHT_ELBOW',
    'LEFT_WRIST',
    'RIGHT_WRIST',
    'LEFT_HIP',
    'RIGHT_HIP',
    'LEFT_KNEE',
    'RIGHT_KNEE',
    'LEFT_ANKLE',
    'RIGHT_ANKLE',
    'LEFT_HEEL',
    'RIGHT_HEEL',
    'LEFT_FOOT_INDEX',
    'RIGHT_FOOT_INDEX',
]

headers = ['label']

# Generate all columns of the data frame
for landmark in landmarks:
    headers += [f'{landmark.lower()}_x', f'{landmark.lower()}_y', f'{landmark.lower()}_z', f'{landmark.lower()}_v']
print(f'Number of columns: {len(headers)}')

empty_df = pd.DataFrame(columns=headers)

Number of columns: 69


In [151]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

In [152]:
def create_image_data(files, features, label, df):
    df = df.copy()

    for file in files:
        image = mp.Image.create_from_file(file)

        pose_landmarks = detector.detect(image).pose_landmarks[0]
        # print('Pose landmarks:', pose_landmarks)
        # print('Number of Pose landmarks:', len(pose_landmarks)) # should be 33

        landmarks = []
        for feature in features:
            landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks = list(np.array(landmarks).flatten())
        # print('Number of landmarks:', len(landmarks))

        # Add the label to the landmarks
        sample = [label] + landmarks
        sample = pd.Series(sample, index=df.columns)

        # Add the sample to the data frame
        df = pd.concat([df, sample.to_frame().T], ignore_index=True)

    return df

In [155]:
def create_video_data(files, frame_capture_rate, features, label, df):
    df = df.copy()
    count =0
    for frame_capture in frame_capture_rate:
      for file in files:
          if file[-4:] ==".mp4":
            for time in range(2):
              frame_number = 0
              print(file)

              with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                  video = cv2.VideoCapture(file)

                  while video.isOpened():
                      ret, frame = video.read()

                      if not ret:
                          break
                      if time == 1:
                        cv2.flip(frame,1)
                      if frame_number % frame_capture == 0:
                          landmarks = []
                          pose_landmarks = pose.process(frame).pose_landmarks.landmark

                          for feature in features:
                              landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
                              landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])

                          landmarks = list(np.array(landmarks).flatten())
                          sample = [count] + landmarks

                          if len(sample) == len(df.columns):
                              sample = pd.Series(sample, index=df.columns)
                              df = pd.concat([df, sample.to_frame().T], ignore_index=True)
                          else:
                            print(count)
                      frame_number += 1

                  video.release()
              count += 1
    return df

# **Generate Data from Images**

In [22]:
image_data_path = 'data/plank/image'
video_data_path = 'data'

In [23]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [24]:
correct_files = [os.path.join(image_data_path, 'correct', file) for file in os.listdir(os.path.join(image_data_path, 'correct'))]
print('Number of correct samples:', len(correct_files))

correct_df = create_image_data(correct_files, landmarks, 'c', empty_df)
print('Correct data frame shape:', correct_df.shape)
correct_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/plank/image/correct'

In [ ]:
low_image_files = [os.path.join(image_data_path, 'low', file) for file in os.listdir(os.path.join(image_data_path, 'low'))]
print('Number of low samples:', len(low_image_files))
print(low_image_files)

low_image_df = create_image_data(low_image_files, landmarks, 'l', empty_df)
print('Low data frame shape:', low_image_df.shape)
low_image_df.head()

In [ ]:
high_image_files = [os.path.join(image_data_path, 'high', file) for file in os.listdir(os.path.join(image_data_path, 'high'))]
print('Number of high samples:', len(high_image_files))

high_image_df = create_image_data(high_image_files, landmarks, 'h', empty_df)
print('High data frame shape:', high_image_df.shape)
high_image_df.head()

# **Generate Data from Videos**

In [156]:
correct_video_files = [os.path.join(video_data_path, 'Good', file) for file in os.listdir(os.path.join(video_data_path, 'Good'))]
print('Number of correct samples:', len(correct_video_files))

correct_video_df = create_video_data(correct_video_files, [5,7,11], landmarks, 'c', empty_df)
print('Correct video data frame shape:', correct_video_df.shape)
correct_video_df.head()


Number of correct samples: 14
data/Good/squat_29.mp4
data/Good/squat_29.mp4
data/Good/Recording 2024-06-11 121938.mp4
data/Good/Recording 2024-06-11 121938.mp4
data/Good/Untitled video - Made with Clipchamp.mp4
data/Good/Untitled video - Made with Clipchamp.mp4
data/Good/Recording 2024-06-11 121800.mp4
data/Good/Recording 2024-06-11 121800.mp4
data/Good/squat_26.mp4
data/Good/squat_26.mp4
data/Good/Untitled video - Made with Clipchamp (1).mp4
data/Good/Untitled video - Made with Clipchamp (1).mp4
data/Good/Recording 2024-06-11 120009.mp4
data/Good/Recording 2024-06-11 120009.mp4
data/Good/Recording 2024-06-11 123509.mp4
data/Good/Recording 2024-06-11 123509.mp4
data/Good/Recording 2024-06-11 120212.mp4
data/Good/Recording 2024-06-11 120212.mp4
data/Good/Recording 2024-06-11 014448.mp4
data/Good/Recording 2024-06-11 014448.mp4
data/Good/Recording 2024-06-11 120511.mp4
data/Good/Recording 2024-06-11 120511.mp4
data/Good/Untitled video - Made with Clipchamp (2).mp4
data/Good/Untitled vide

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,0.0,0.479050,0.328261,0.073631,0.993453,0.419109,0.376748,0.089880,0.996429,0.460490,...,-0.134987,0.965483,0.443271,0.795955,0.204752,0.858785,0.494768,0.850146,-0.174473,0.983856
1,0.0,0.481488,0.368312,0.066977,0.994051,0.421666,0.405514,0.065365,0.996756,0.472053,...,-0.033204,0.967483,0.440118,0.798217,0.325332,0.857009,0.491883,0.850167,-0.046242,0.984811
2,0.0,0.482931,0.427831,0.057195,0.994582,0.421800,0.462956,0.072406,0.997066,0.475069,...,-0.078822,0.968924,0.439564,0.799258,0.336695,0.848977,0.487014,0.857961,-0.081337,0.985481
3,0.0,0.486038,0.483727,0.032684,0.994856,0.423156,0.509967,0.033593,0.997254,0.474153,...,-0.074299,0.969738,0.443394,0.813184,0.304532,0.839132,0.485744,0.869231,-0.069839,0.985611
4,0.0,0.473576,0.509812,-0.079287,0.994416,0.480399,0.575959,0.013989,0.996789,0.431148,...,-0.141395,0.946827,0.457386,0.861279,-0.114541,0.839098,0.482658,0.868733,-0.174141,0.966884


In [157]:
high_video_files = [os.path.join(video_data_path, 'Bad', file) for file in os.listdir(os.path.join(video_data_path, 'Bad'))]
print('Number of high samples:', len(high_video_files))

high_video_df = create_video_data(high_video_files, [5,7,11], landmarks, 'h', empty_df)
print('High video data frame shape:', high_video_df.shape)
high_video_df.head()

Number of high samples: 8
data/Bad/Recording 2024-06-11 020741.mp4
data/Bad/Recording 2024-06-11 020741.mp4
data/Bad/Recording 2024-06-11 020114.mp4
data/Bad/Recording 2024-06-11 020114.mp4
data/Bad/Untitled video - Made with Clipchamp.mp4
data/Bad/Untitled video - Made with Clipchamp.mp4
data/Bad/Recording 2024-06-11 020524.mp4
data/Bad/Recording 2024-06-11 020524.mp4
data/Bad/Recording 2024-06-11 121858.mp4
data/Bad/Recording 2024-06-11 121858.mp4
data/Bad/Recording 2024-06-11 014614.mp4
data/Bad/Recording 2024-06-11 014614.mp4
data/Bad/Recording 2024-06-11 023751.mp4
data/Bad/Recording 2024-06-11 023751.mp4
data/Bad/Recording 2024-06-11 020741.mp4
data/Bad/Recording 2024-06-11 020741.mp4
data/Bad/Recording 2024-06-11 020114.mp4
data/Bad/Recording 2024-06-11 020114.mp4
data/Bad/Untitled video - Made with Clipchamp.mp4
data/Bad/Untitled video - Made with Clipchamp.mp4
data/Bad/Recording 2024-06-11 020524.mp4
data/Bad/Recording 2024-06-11 020524.mp4
data/Bad/Recording 2024-06-11 121858

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,0.0,0.336524,0.251659,-0.228495,0.999590,0.485917,0.291467,-0.510621,0.999709,0.516302,...,1.161384,0.782559,0.435137,0.878558,-0.298376,0.980801,0.423135,0.772580,1.138791,0.883147
1,0.0,0.336125,0.249132,-0.254893,0.999608,0.494472,0.290933,-0.498463,0.999721,0.503199,...,1.201425,0.772868,0.437666,0.877782,-0.368390,0.980336,0.431112,0.773484,1.148647,0.876296
2,0.0,0.335988,0.247955,-0.263185,0.999624,0.500434,0.290363,-0.489897,0.999730,0.495886,...,1.205658,0.764560,0.439210,0.877277,-0.391559,0.979869,0.433907,0.773936,1.149068,0.870342
3,0.0,0.335759,0.247155,-0.269233,0.999641,0.504030,0.290037,-0.486833,0.999738,0.491602,...,1.203968,0.758460,0.440074,0.876987,-0.399434,0.979481,0.435170,0.774221,1.148638,0.866098
4,0.0,0.341108,0.252667,-0.475493,0.999674,0.554779,0.299014,-0.421594,0.999761,0.443241,...,1.329596,0.767642,0.440875,0.876795,-0.621046,0.980276,0.439390,0.774738,1.207919,0.871249


In [158]:
print(set(correct_video_df[high_video_df.columns[0]]))
print(set(high_video_df[high_video_df.columns[0]].values))
# Convert the DataFrame to a NumPy array directly, without grouping
timed_correct_array = correct_video_df.to_numpy()

# If you need to work with the groups separately, consider a list or dictionary
concatinated_correct = []
current_np = []
current_number = 0
for row in timed_correct_array:

  if row[0] !=current_number:
    current_number = row[0]
    concatinated_correct.append(["c"]+current_np)
    current_np = []
  for a in row[1:]:
    current_np.append(a)
concatinated_correct.append(["c"]+current_np)



# Convert the DataFrame to a NumPy array directly, without grouping
timed_wrong_array = high_video_df.to_numpy()

# If you need to work with the groups separately, consider a list or dictionary
concatinated_wrong = []
current_np = []
current_number = 0
for row in timed_wrong_array:
  if row[0] !=current_number:
    current_number = row[0]
    concatinated_wrong.append(["w"]+current_np)
    current_np = []
  for a in row[1:]:
    current_np.append(a)
concatinated_wrong.append(["w"]+current_np)
print(len(concatinated_correct))
print(len(concatinated_wrong))
combined = concatinated_correct +concatinated_wrong
print(len(combined))
maxLength = max(len(x) for x in combined)
lengthened = []
for a in combined:
  lengthened.append([[a[0]]+[0]*(maxLength-len(a))+list(a[1:])])
print(lengthened)
lengthened = [a[0] for a in lengthened]
print(max(len(x) for x in lengthened))
print(min(len(x) for x in lengthened))
#print(np.pad(timed_wrong_groups,mode='maximum'))
'''timed_correct_groups
plank_data = pd.read_csv('data/plank/plank_data.csv')

plank_data['label'] = plank_data['label'].str.lower()

# Separate the classes

bad_data = plank_data[plank_data['label'] == 'h']
correct_data = plank_data[plank_data['label'] == 'c']

# Randomly sample 100 from each class

sample_size = 200
sample_low = low_data.sample(n=sample_size, random_state=42)
sample_high = high_data.sample(n=sample_size, random_state=42)
sample_correct = correct_data.sample(n=sample_size, random_state=42)

sampled_data = pd.concat([sample_low, sample_high, sample_correct])
print('Sampled data shape:', sampled_data.shape)
sampled_data.head()'''

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



"timed_correct_groups\nplank_data = pd.read_csv('data/plank/plank_data.csv')\n\nplank_data['label'] = plank_data['label'].str.lower()\n\n# Separate the classes\n\nbad_data = plank_data[plank_data['label'] == 'h']\ncorrect_data = plank_data[plank_data['label'] == 'c']\n\n# Randomly sample 100 from each class\n\nsample_size = 200\nsample_low = low_data.sample(n=sample_size, random_state=42)\nsample_high = high_data.sample(n=sample_size, random_state=42)\nsample_correct = correct_data.sample(n=sample_size, random_state=42)\n\nsampled_data = pd.concat([sample_low, sample_high, sample_correct])\nprint('Sampled data shape:', sampled_data.shape)\nsampled_data.head()"

# **Combine all data**

In [159]:
with open("data.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile)
     for a in lengthened:
       wr.writerow(a)


In [ ]:
final_data.to_csv('data/plank/train_data.csv', index=False)

NameError: name 'final_data' is not defined

In [2]:
!pip install mediapipe opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [13]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task